In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from bs4 import BeautifulSoup


In [5]:
# scrape linkedin profile and extract education, experience, skills, etc. in a structured format

def scrape_linkedin_profile(linkedin_url):
    response = requests.get(linkedin_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup.prettify())

linkedin_url = 'https://www.linkedin.com/in/yanis-miraoui-54377a1b8/'
scrape_linkedin_profile(linkedin_url)

<html>
 <head>
  <script type="text/javascript">
   window.onload = function() {
  // Parse the tracking code from cookies.
  var trk = "bf";
  var trkInfo = "bf";
  var cookies = document.cookie.split("; ");
  for (var i = 0; i < cookies.length; ++i) {
    if ((cookies[i].indexOf("trkCode=") == 0) && (cookies[i].length > 8)) {
      trk = cookies[i].substring(8);
    }
    else if ((cookies[i].indexOf("trkInfo=") == 0) && (cookies[i].length > 8)) {
      trkInfo = cookies[i].substring(8);
    }
  }

  if (window.location.protocol == "http:") {
    // If "sl" cookie is set, redirect to https.
    for (var i = 0; i < cookies.length; ++i) {
      if ((cookies[i].indexOf("sl=") == 0) && (cookies[i].length > 3)) {
        window.location.href = "https:" + window.location.href.substring(window.location.protocol.length);
        return;
      }
    }
  }

  // Get the new domain. For international domains such as
  // fr.linkedin.com, we convert it to www.linkedin.com
  // treat .cn similar 

In [3]:
# scrape linkedin profile and extract education, experience, skills, etc. in a structured format

def scrape_github_profile(linkedin_url):
    response = requests.get(linkedin_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # select only after application-main class
    content = soup.find('div', class_='application-main')
    print(content.prettify())

github_url = 'https://github.com/yanismiraoui'
scrape_github_profile(github_url)

<div class="application-main" data-commit-hovercards-enabled="" data-discussion-hovercards-enabled="" data-issue-and-pr-hovercards-enabled="">
 <main>
  <x-banner data-dismiss-scheme="none" data-view-component="true">
   <div class="js-user-status-banner Banner flash Banner--full flash-full" data-view-component="true" hidden="hidden">
    <div class="Banner-visual">
     <svg aria-hidden="true" class="octicon octicon-info" data-view-component="true" height="16" version="1.1" viewbox="0 0 16 16" width="16">
      <path d="M0 8a8 8 0 1 1 16 0A8 8 0 0 1 0 8Zm8-6.5a6.5 6.5 0 1 0 0 13 6.5 6.5 0 0 0 0-13ZM6.5 7.75A.75.75 0 0 1 7.25 7h1a.75.75 0 0 1 .75.75v2.75h.25a.75.75 0 0 1 0 1.5h-2a.75.75 0 0 1 0-1.5h.25v-2h-.25a.75.75 0 0 1-.75-.75ZM8 6a1 1 0 1 1 0-2 1 1 0 0 1 0 2Z">
      </path>
     </svg>
    </div>
    <div class="Banner-message" data-view-component="true">
     <p class="Banner-title" data-target="x-banner.titleText">
      <span aria-live="polite" class="js-user-status-banner-mes

In [31]:
# use pymupdf to read cv and extract text in a structured format
import pymupdf

def extract_text_from_cv(cv_path):
    pdf = pymupdf.open(cv_path)
    text = ''
    for page in pdf.pages():
        text += page.get_text()
    return text

extract_text_from_cv('./cv_examples/CV Yanis MIRAOUI.pdf')


# use llm to structure the text extracted from the cv (using together ai)
from together import Together
import os
import toml

secrets_path = 'secrets.toml'
secrets = toml.load(secrets_path)
os.environ['TOGETHER_API_KEY'] = secrets['TOGETHER_API_KEY']

client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

def structured_cv_text(text):
    prompt = """
        Extract the following information from the text: 
        {text}

        
        The output should be a structured json with the following fields:
        - education: list of tuples (degree, school, location, start date, end date)
        - experience: list of tuples (company, title, location, start date, end date)
        - skills: list of strings
        - projects: list of tuples (name, description, start date, end date)
        - languages: list of strings
        If you cannot find a field, leave it empty. Make sure that the information is correct, present in the text and not hallucinated.
        """       
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt.format(text=text)}],
        stream=False
    )
    return response.choices[0].message.content

In [32]:
extract_text_from_cv('./cv_examples/CV Yanis MIRAOUI.pdf')

'Yanis MIRAOUI \nStanford, USA | +1 (650) 505 3509 | ymiraoui@stanford.edu | https://yanis.miraoui.com/ \nEDUCATION \nStanford University \nStanford, USA \nMS Statistics & Data Science \nSeptember 2023 - June 2025 \nImperial College London & ETH Zürich \nLondon, UK – Zürich, Switzerland \nMSci Mathematics, Grade: First-Class Honors \nOctober 2019 - June 2023 \nLANGUAGES\nFrench: Native  \n \nMoroccan: Fluent \nEnglish: Fluent \nGerman: Professional proficiency  \nSpanish: Fluent \nArabic: Intermediate\nPROFESSIONAL EXPERIENCES \nOracle \nRedwood City, USA \nResearch Data Scientist Intern \nSeptember 2024 - Present \n• \nDesigned, implemented, and evaluated fully integrated LLM Agents at Oracle Labs, improving workflow automation for data \nscientists \n• \nCollaborated with cross-functional teams to identify optimization opportunities, leveraging LLMs to improve agent accuracy \nStanford Computational Neuroscience Laboratory (CNS Lab) \n \nStanford, USA \nResearch Assistant \n \nSeptem

In [33]:
print(structured_cv_text(extract_text_from_cv('./cv_examples/CV Yanis MIRAOUI.pdf')))

id='8daecc75ff0715e9-SJC' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1730326889 model='meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=14713538458809217000, finish_reason=<FinishReason.EOS: 'eos'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='```json\n{\n    "education": [\n        ("MS Statistics & Data Science", "Stanford University", "Stanford, USA", "September 2023", "June 2025"),\n        ("MSci Mathematics", "Imperial College London & ETH Zürich", "London, UK – Zürich, Switzerland", "October 2019", "June 2023")\n    ],\n    "experience": [\n        ("Oracle", "Research Data Scientist Intern", "Redwood City, USA", "September 2024", "Present"),\n        ("Stanford Computational Neuroscience Laboratory (CNS Lab)", "Research Assistant", "Stanford, USA", "September 2023", "Present"),\n        ("Snowflake", "Applied Research Data Scientist Intern", "San Mateo, USA", "Ju